In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/Shared drives/Тяжелые проекты/ИАД/intro-to-dl-seminars/hw_5_text')

In [3]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, Conv1D, LeakyReLU,BatchNormalization, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.optimizers import Adamax, Adam
from tqdm import tqdm_notebook

In [5]:
tf.test.gpu_device_name()

'/device:GPU:0'

По результатам экспериментов, лучше себя показывает эмбединг, обученный на твиттере

In [0]:
EMBEDDING_FILE='glove/glove.twitter.27B.100d.txt'
TRAIN_DATA_FILE='x_train.txt'
TEST_DATA_FILE='x_test.txt'
TRAIN_LABELS='y_train.csv'

In [7]:
!cat {EMBEDDING_FILE} | wc -l

1193514


In [0]:
embed_size = 100 # Размерность эмбединга
max_features = 50000 # Максимальное количество фич
maxlen = 150 # Максимальная длина фразы в токенах

In [9]:
train = pd.read_csv(TRAIN_DATA_FILE, header=None, names=['text'], sep="ЯЯЯЯЯЯЯЯ")
test = pd.read_csv(TEST_DATA_FILE, header=None, names=['text'], sep="ЯЯЯЯЯЯЯЯ")
y = pd.read_csv(TRAIN_LABELS)['Probability'].values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [0]:
list_sentences_train = train["text"].fillna("_na_").values
list_sentences_test = test["text"].fillna("_na_").values

In [11]:
%%time
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

CPU times: user 7min 47s, sys: 2.86 s, total: 7min 50s
Wall time: 7min 50s


Как можно заметить, данные в векторах не соответствуют тому формату, который был подготовлен

In [12]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
with open(EMBEDDING_FILE) as f:
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in tqdm_notebook(f))

In [0]:
sh = embeddings_index['a'].shape

In [14]:
for k, v in embeddings_index.items():
    if v.shape != sh:
        print(k)
        

-0.32053


In [0]:
del embeddings_index['-0.32053']

In [16]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.02631098, 0.58370703)

In [0]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

Сводная таблица результата обучения простой CNN на 10% данных (эмбединг, Conv1D, GlobalMaxPooling, Dense).

Чем больше фильтров, тем лучше модель, но размер кернела лучше взять средним.

| val_AUC |   32   |   64   |   128  |   256  |
|:-------:|:------:|:------:|:------:|:------:|
|    3    | 0.9678 | 0.9712 | 0.9732 | 0.9754 |
|    5    | 0.9704 | 0.9720 | 0.9741 | 0.9758 |
|    6    | 0.9694 | 0.9713 | 0.9738 | 0.9758 |


Далее возьмем 2 разноприродные модели, которые показали наилучший результат и объединим их.

Так же была попытка добавить еще 1 вход, основанный на метаданных текста (количество слов, символов, уникальных слов и т.д.), но такая модель показывала неудовлетворительное качество

In [0]:
def cnn_layers(inp):
    x = Embedding(max_features, embed_size, input_length=maxlen, weights=[embedding_matrix], trainable = True)(inp)

    x = Conv1D(filters=64, kernel_size=5)(x)
    x = LeakyReLU(0.1)(x)
    x = Conv1D(filters=64, kernel_size=5)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)  
    x = MaxPooling1D(padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv1D(filters=128, kernel_size=5)(x)
    x = LeakyReLU(0.1)(x)
    x = Conv1D(filters=128, kernel_size=5)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)  
    x = MaxPooling1D(padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv1D(filters=256, kernel_size=5)(x)
    x = LeakyReLU(0.1)(x)
    x = Conv1D(filters=256, kernel_size=5)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)  
    x = MaxPooling1D(padding='same')(x)
    x = Dropout(0.2)(x)

    x = Flatten()(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.1)(x)  
    return x

In [0]:
def lstm_layers(inp):
    x = Embedding(max_features, embed_size, input_length=maxlen, weights=[embedding_matrix], trainable = True)(inp)
    x = Bidirectional(LSTM(80, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
    gmx = GlobalMaxPool1D()(x)
    gax = GlobalAveragePooling1D()(x)
    concatenated = concatenate([gmx, gax])
    x = Dense(80, activation="relu")(concatenated)
    return x

In [0]:
input_text = Input(shape=(maxlen,))

cnn = cnn_layers(input_text)
lstm = lstm_layers(input_text)
concatenated = concatenate([cnn, lstm])

x = Dropout(0.1)(concatenated)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=input_text, outputs=x)
model.compile(loss='binary_crossentropy', optimizer=Adam(clipvalue=2, clipnorm=2), metrics=['accuracy' ,'AUC'])

In [0]:
model=keras.models.load_model()

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 150, 100)     5000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 146, 64)      32064       embedding[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 146, 64)      0           conv1d[0][0]                     
______________________________________________________________________________________________

In [0]:
class ModelSaveCallback(keras.callbacks.Callback):

    def __init__(self, file_name):
        super(ModelSaveCallback, self).__init__()
        self.file_name = file_name

    def on_epoch_end(self, epoch, logs=None):
        filename = self.file_name.format(epoch)
        keras.models.save_model(self.model, filename)
class ModelPredictCallback(keras.callbacks.Callback):

    def __init__(self, file_name):
        super(ModelPredictCallback, self).__init__()
        self.file_name = file_name

    def on_epoch_end(self, epoch, logs=None):
        filename = self.file_name.format(epoch)
        result = self.model.predict(X_test, batch_size=batch_size*16, verbose=0)
        df = pd.DataFrame()
        df['Id'] = range(1, len(X_test) + 1)
        df['Probability'] = result
        df.to_csv(filename, index=None)

In [0]:
TAKE=28 # Номер попытки
batch_size=512
NAME='kernel_conv_lstm_nofeatures_glove.twitter.27B.100d'
model_filename = '{0:02d}_{1:s}_{{0:02d}}.hdf5'.format(TAKE, NAME)
predict_filename = '{0:02d}_{1:s}_{{0:02d}}.csv'.format(TAKE, NAME)

In [0]:
model.fit(X_train, y, batch_size=batch_size, epochs=1, validation_split=0.1,verbose=1,
          initial_epoch=0,
        callbacks = [
            ModelSaveCallback(model_filename),
            ModelPredictCallback(predict_filename),
        ],
        )

In [34]:
from IPython.display import Audio
Audio('First_epoch.mp3', autoplay=True) 

In [42]:
model.fit(X_train, y, batch_size=batch_size, epochs=7, validation_split=0.1,verbose=2,
          initial_epoch=1,
        callbacks = [
            ModelSaveCallback(model_filename),
            ModelPredictCallback(predict_filename),
        ],
        )

Train on 3240000 samples, validate on 360000 samples
Epoch 2/7
3240000/3240000 - 3480s - loss: 0.1211 - accuracy: 0.9559 - AUC: 0.9900 - val_loss: 0.1228 - val_accuracy: 0.9549 - val_AUC: 0.9899
Epoch 3/7
3240000/3240000 - 3433s - loss: 0.1052 - accuracy: 0.9628 - AUC: 0.9921 - val_loss: 0.1213 - val_accuracy: 0.9563 - val_AUC: 0.9899
Epoch 4/7


KeyboardInterrupt: ignored

Наша модель полностью обучилась, добавим элемента неожиданности

In [54]:
for i in range(20):
    print(i)
    idx = np.random.randint(X_train.shape[0], size=int(X_train.shape[0]/20))
    X_train1 = X_train[idx, :]
    y1 = y[idx]
    model.fit(X_train1, y1, batch_size=int(batch_size/2), epochs=i+4,verbose=1, initial_epoch=i+3)

0
Train on 180000 samples
Epoch 4/4
180000/180000 [==============================] - 365s 2ms/sample - loss: 0.0925 - accuracy: 0.9684 - AUC: 0.9936
1
Train on 180000 samples
Epoch 5/5
180000/180000 [==============================] - 365s 2ms/sample - loss: 0.0906 - accuracy: 0.9683 - AUC: 0.9939
2
Train on 180000 samples
Epoch 6/6
180000/180000 [==============================] - 362s 2ms/sample - loss: 0.0891 - accuracy: 0.9696 - AUC: 0.9940
3
Train on 180000 samples
Epoch 7/7
180000/180000 [==============================] - 357s 2ms/sample - loss: 0.0883 - accuracy: 0.9693 - AUC: 0.9941
4
Train on 180000 samples
Epoch 8/8
180000/180000 [==============================] - 362s 2ms/sample - loss: 0.0876 - accuracy: 0.9701 - AUC: 0.9942
5
Train on 180000 samples
Epoch 9/9
180000/180000 [==============================] - 365s 2ms/sample - loss: 0.0906 - accuracy: 0.9692 - AUC: 0.9938
6
Train on 180000 samples
Epoch 10/10
180000/180000 [==============================] - 365s 2ms/sample - l

In [0]:
result = model.predict(X_test, batch_size=batch_size*16, verbose=0)
df = pd.DataFrame()
df['Id'] = range(1, len(X_test) + 1)
df['Probability'] = result
df.to_csv('night_sgd.csv', index=None)

In [0]:
model.fit(X_train, y, batch_size=batch_size, epochs=3, validation_split=0.1,verbose=1,
          initial_epoch=0,
        callbacks = [
            ModelSaveCallback(model_filename),
            ModelPredictCallback(predict_filename),
        ],
        )

Train on 3240000 samples, validate on 360000 samples
Epoch 1/3
1728000/3240000 [===============>..............] - ETA: 26:15 - loss: 0.0762 - accuracy: 0.9743 - AUC: 0.9953